## Analizando la evaluación Lazy de Spark

El objetivo de esta libreta es analizar y comprender la evaluación Lazy que realiza Spark. A través de algunos escenarios revisaremos en qué consiste.

In [0]:
#Creado dataFrame
df_fifa = spark.read.csv('/FileStore/tables/players_20.csv', header = True, inferSchema = True)

df_fifa = df_fifa.select('short_name','age')

In [0]:
df_fifa.show(5)

+-----------------+---+
|       short_name|age|
+-----------------+---+
|         L. Messi| 32|
|Cristiano Ronaldo| 34|
|        Neymar Jr| 27|
|         J. Oblak| 26|
|        E. Hazard| 28|
+-----------------+---+
only showing top 5 rows



In [0]:
#Escenario 1
df_fifa.withColumn('edad_meses', df_fifa.age * 12).show(5)

+-----------------+---+----------+
|       short_name|age|edad_meses|
+-----------------+---+----------+
|         L. Messi| 32|       384|
|Cristiano Ronaldo| 34|       408|
|        Neymar Jr| 27|       324|
|         J. Oblak| 26|       312|
|        E. Hazard| 28|       336|
+-----------------+---+----------+
only showing top 5 rows



In [0]:
#Escenario 2
df_fifa.withColumn('edad_dias', df_fifa.age * 365)
df_fifa.drop('edad_dias')
df_fifa.show(5)

#Drop the column “edad_dias”.
#Print the output of the dataframe.

+-----------------+---+
|       short_name|age|
+-----------------+---+
|         L. Messi| 32|
|Cristiano Ronaldo| 34|
|        Neymar Jr| 27|
|         J. Oblak| 26|
|        E. Hazard| 28|
+-----------------+---+
only showing top 5 rows



En el escenario 2 observamos que las ventajas de Spark Lazy. Es decir, Spark se da cuenta de que la creación de edad_dias no tiene valor e ignora por completo ese paso. Y debido a esta "pereza", el trabajo corre más rápido.

## Observemos otro ejemplo

In [0]:
#1) Creamos una lista
import numpy as np
my_list = [i for i in range(1,10000000)]

In [0]:
#Creamos un RDD a partir del array de datos
rdd_data_0 = sc.parallelize(my_list,4)
print("Número de particiones: ", rdd_data_0.getNumPartitions())
rdd_data_0

Número de particiones:  4
Out[6]: ParallelCollectionRDD[351] at readRDDFromInputStream at PythonRDD.scala:413

In [0]:
# Aplicamos una transformación básica: sumamos 5 a cada elemento del RDD
rdd_data_1 = rdd_data_0.map(lambda x : x+5) #en este punto Spark, no ha iniciado ninguna transformación, 
                                            #solo registra una serie de transformaciones en la forma linaje RDD.
# RDD object
print(rdd_data_1)

#debugging
rdd_data_1.toDebugString()  #observando el linaje RDD

PythonRDD[352] at RDD at PythonRDD.scala:58
Out[7]: b'(4) PythonRDD[352] at RDD at PythonRDD.scala:58 []\n |  ParallelCollectionRDD[351] at readRDDFromInputStream at PythonRDD.scala:413 []'

Observamos que PythonRDD[352] está conectado a ParallelCollectionRDD[351]

In [0]:
#Una vez más sumamos valores al RDD
rdd_data_2 = rdd_data_1.map(lambda x : x+20)

# RDD Object
print(rdd_data_2)

# Conseguimos el linaje
print(rdd_data_2.toDebugString())

PythonRDD[353] at RDD at PythonRDD.scala:58
b'(4) PythonRDD[353] at RDD at PythonRDD.scala:58 []\n |  ParallelCollectionRDD[351] at readRDDFromInputStream at PythonRDD.scala:413 []'


Observamos que PythonRDD[353] está conectado a ParallelCollectionRDD[351].
Podemos ver que automáticamente se ha saltado un paso redundante y agregará 25 en un solo paso en lugar de cómo lo definimos. Entonces, Spark define automáticamente la mejor ruta para realizar cualquier acción y solo realiza las transformaciones cuando es necesario.

### Ventajas de la evaluación "lazy"
- Reduce la complejidad: Las dos complejidades principales de cualquier operación son el tiempo y del espacio. Usando la evaluación perezosa de Apache Spark podemos superar ambos. Como no ejecutamos todas las operaciones, por lo tanto, se ahorra tiempo. Nos permite trabajar con una estructura de datos infinita. La acción se activa solo cuando se requieren los datos, reduce la sobrecarga.

- Optimización de recursos: la optimización se logra a através de reducir el número de recursos a emplear.

- Ahorro de cómputo e incrementa la velocidad: la evaluación perezosa juega un papel clave en el ahorro de gastos generales de cálculo. Dado que solo los valores necesarios se calculan. Ahorra el viaje entre el conductor y el grupo, por lo que acelera el proceso.


### Ejercicio


a) Aplica 10 funciones lambda sumando 200 (rdd_data_0.map(lambda x : x+200)) e imprime el resultado posteriormente.

b) Aplica 1 función lambda sumando 2000 (rdd_data_0.map(lambda x : x+2000)) e imprime el resultado posteriormente.

c) Mide el tiempo de ejecución para a) y b). Discute brevemente los resultados.
